In [72]:
import pandas as pd
import numpy as np
import MeCab
import urllib.request

# pandasの表示設定を変更
pd.set_option('display.max_rows', None) #行の表示上限をなくす
pd.set_option('display.max_columns', None) #列の表示上限をなくす
pd.set_option('display.max_colwidth', None) #列の内容の表示上限をなくす

In [60]:
odaiba_df = pd.read_csv('csv/odaiba_reviews.csv', encoding='utf-8-sig')
odaiba_df.head()

,shop_name,shop_url,review_text
0,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),https://tabelog.com/tokyo/A1313/A131306/13251352/,平日のランチブッフェに行きました。店内は広々していて、テーブルもイスも大きめで、ゆったり過ごせました。季節の料理がメインで、楽しく選ぶことができ、あまり混んでいなかったからか、並ぶこと...
1,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),https://tabelog.com/tokyo/A1313/A131306/13251352/,GW中にレストラン利用のみで訪問。ホテルの中にあるため、エントランス、トイレなどを含めて全体的に綺麗で快適。今年はどうやら5周年の模様。料理の品数は可もなく不可もなく感じるが、一つ一つ丁寧...
2,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),https://tabelog.com/tokyo/A1313/A131306/13251352/,素敵なレストランでした️ライブキッチンでローストビーフとスープを頂けました。海鮮も置いてあり自由に丼が作れます。料理も充実していてとても良かったです。デザートにフルーツやケーキ...
3,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),https://tabelog.com/tokyo/A1313/A131306/13251352/,この日は早朝から丸一日展示会を回る予定にしていたこともあって、夕食はホテルのビュッフェを予約していました。夕方から荒天になって屋外はどんどん風雨が強くなっていたので、予約をしていて良かったなと...
4,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),https://tabelog.com/tokyo/A1313/A131306/13251352/,一休予約で100円引き！コスパいい！ホテルビュッフェは、旅行に行った気分になる！有明で降りたのは、はじめてで、楽しい！イチゴのモンブラン！自分で作る海鮮丼！桜...


## 口コミ情報

In [61]:
review_df = odaiba_df[['shop_name', 'review_text']]
review_df.head()

,shop_name,review_text
0,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),平日のランチブッフェに行きました。店内は広々していて、テーブルもイスも大きめで、ゆったり過ごせました。季節の料理がメインで、楽しく選ぶことができ、あまり混んでいなかったからか、並ぶこと...
1,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),GW中にレストラン利用のみで訪問。ホテルの中にあるため、エントランス、トイレなどを含めて全体的に綺麗で快適。今年はどうやら5周年の模様。料理の品数は可もなく不可もなく感じるが、一つ一つ丁寧...
2,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),素敵なレストランでした️ライブキッチンでローストビーフとスープを頂けました。海鮮も置いてあり自由に丼が作れます。料理も充実していてとても良かったです。デザートにフルーツやケーキ...
3,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),この日は早朝から丸一日展示会を回る予定にしていたこともあって、夕食はホテルのビュッフェを予約していました。夕方から荒天になって屋外はどんどん風雨が強くなっていたので、予約をしていて良かったなと...
4,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),一休予約で100円引き！コスパいい！ホテルビュッフェは、旅行に行った気分になる！有明で降りたのは、はじめてで、楽しい！イチゴのモンブラン！自分で作る海鮮丼！桜...


In [62]:
# 欠損値を確認する関数
def missing_values_table(df): 
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum()/len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    return mis_val_table_ren_columns 

missing_values_table(review_df)

,Missing Values,% of Total Values
shop_name,0,0.0
review_text,0,0.0


In [63]:
# あまり関係のないと思われる数字を全て0に置き換える関数
import re
def replace_number_to_zero(text):
    changed_text = re.sub(r'[0-9]+', "0", text) #半角
    changed_text = re.sub(r'[0-9]+', "0", changed_text) #全角
    return changed_text

# 数字を0に置換
review_df['review_number_to_zero'] = review_df['review_text'].map(replace_number_to_zero)
review_df.head()

,shop_name,review_text,review_number_to_zero
0,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),平日のランチブッフェに行きました。店内は広々していて、テーブルもイスも大きめで、ゆったり過ごせました。季節の料理がメインで、楽しく選ぶことができ、あまり混んでいなかったからか、並ぶこと...,平日のランチブッフェに行きました。店内は広々していて、テーブルもイスも大きめで、ゆったり過ごせました。季節の料理がメインで、楽しく選ぶことができ、あまり混んでいなかったからか、並ぶこと...
1,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),GW中にレストラン利用のみで訪問。ホテルの中にあるため、エントランス、トイレなどを含めて全体的に綺麗で快適。今年はどうやら5周年の模様。料理の品数は可もなく不可もなく感じるが、一つ一つ丁寧...,GW中にレストラン利用のみで訪問。ホテルの中にあるため、エントランス、トイレなどを含めて全体的に綺麗で快適。今年はどうやら0周年の模様。料理の品数は可もなく不可もなく感じるが、一つ一つ丁寧...
2,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),素敵なレストランでした️ライブキッチンでローストビーフとスープを頂けました。海鮮も置いてあり自由に丼が作れます。料理も充実していてとても良かったです。デザートにフルーツやケーキ...,素敵なレストランでした️ライブキッチンでローストビーフとスープを頂けました。海鮮も置いてあり自由に丼が作れます。料理も充実していてとても良かったです。デザートにフルーツやケーキ...
3,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),この日は早朝から丸一日展示会を回る予定にしていたこともあって、夕食はホテルのビュッフェを予約していました。夕方から荒天になって屋外はどんどん風雨が強くなっていたので、予約をしていて良かったなと...,この日は早朝から丸一日展示会を回る予定にしていたこともあって、夕食はホテルのビュッフェを予約していました。夕方から荒天になって屋外はどんどん風雨が強くなっていたので、予約をしていて良かったなと...
4,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),一休予約で100円引き！コスパいい！ホテルビュッフェは、旅行に行った気分になる！有明で降りたのは、はじめてで、楽しい！イチゴのモンブラン！自分で作る海鮮丼！桜...,一休予約で0円引き！コスパいい！ホテルビュッフェは、旅行に行った気分になる！有明で降りたのは、はじめてで、楽しい！イチゴのモンブラン！自分で作る海鮮丼！桜...


In [79]:
import MeCab

# 【修正点1】-Owakatiを削除し、デフォルトの出力形式に戻す
# NEologd辞書を使いたい場合は、コメントにあるように -d オプションでパスを指定してください
# tagger = MeCab.Tagger('-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd')
tagger = MeCab.Tagger()

# 分かち書きした結果を返す関数
def leaving_space_between_words_column(text):
    # 欠損値(None, nan)など、文字列でないデータが来た場合は空文字を返す
    if not isinstance(text, str):
        return ''

    words_to_keep = []
    for line in tagger.parse(text).splitlines():
        if line == 'EOS' or line == '':
            continue
        
        parts = line.split('\t')
        if len(parts) == 2:
            surface = parts[0]
            feature = parts[1].split(',')
            part_of_speech = feature[0]
            
            # 【修正点2】除外リストから「動詞」を削除
            if part_of_speech not in ['助詞', '助動詞', '接続詞', '記号']:
                words_to_keep.append(surface)

    return ' '.join(words_to_keep)

# 分かち書きしたカラムをdfに追加する
review_df['review_clearn'] = review_df['review_number_to_zero'].map(leaving_space_between_words_column)
review_df.head()

,shop_name,review_text,review_number_to_zero,lsbw,review_clearn
0,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),平日のランチブッフェに行きました。店内は広々していて、テーブルもイスも大きめで、ゆったり過ごせました。季節の料理がメインで、楽しく選ぶことができ、あまり混んでいなかったからか、並ぶこと...,平日のランチブッフェに行きました。店内は広々していて、テーブルもイスも大きめで、ゆったり過ごせました。季節の料理がメインで、楽しく選ぶことができ、あまり混んでいなかったからか、並ぶこと...,平日 ランチブッフェ 行き 店内 広々 し い テーブル イス 大きめ ゆったり 過ごせ 季節 料理 メイン 楽しく 選ぶ こと でき あまり 混ん い 並ぶ こと ...,平日 ランチブッフェ 行き 店内 広々 し い テーブル イス 大きめ ゆったり 過ごせ 季節 料理 メイン 楽しく 選ぶ こと でき あまり 混ん い 並ぶ こと ...
1,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),GW中にレストラン利用のみで訪問。ホテルの中にあるため、エントランス、トイレなどを含めて全体的に綺麗で快適。今年はどうやら5周年の模様。料理の品数は可もなく不可もなく感じるが、一つ一つ丁寧...,GW中にレストラン利用のみで訪問。ホテルの中にあるため、エントランス、トイレなどを含めて全体的に綺麗で快適。今年はどうやら0周年の模様。料理の品数は可もなく不可もなく感じるが、一つ一つ丁寧...,GW 中 レストラン 利用 訪問 ホテル 中 ある ため エン トランス トイレ 含め 全体 的 綺麗 快適 今年 どうやら 0 周年 模様 料理 品数 可 なく 不可 なく 感じる 一つ 一つ 丁寧 ...,GW 中 レストラン 利用 訪問 ホテル 中 ある ため エン トランス トイレ 含め 全体 的 綺麗 快適 今年 どうやら 0 周年 模様 料理 品数 可 なく 不可 なく 感じる 一つ 一つ 丁寧 ...
2,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),素敵なレストランでした️ライブキッチンでローストビーフとスープを頂けました。海鮮も置いてあり自由に丼が作れます。料理も充実していてとても良かったです。デザートにフルーツやケーキ...,素敵なレストランでした️ライブキッチンでローストビーフとスープを頂けました。海鮮も置いてあり自由に丼が作れます。料理も充実していてとても良かったです。デザートにフルーツやケーキ...,素敵 レストラン ライブ キッチン ロースト ビーフ スープ 頂け 海鮮 置い あり 自由 丼 作れ 料理 充実 し い とても 良かっ デザート フルーツ ケーキ ...,素敵 レストラン ライブ キッチン ロースト ビーフ スープ 頂け 海鮮 置い あり 自由 丼 作れ 料理 充実 し い とても 良かっ デザート フルーツ ケーキ ...
3,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),この日は早朝から丸一日展示会を回る予定にしていたこともあって、夕食はホテルのビュッフェを予約していました。夕方から荒天になって屋外はどんどん風雨が強くなっていたので、予約をしていて良かったなと...,この日は早朝から丸一日展示会を回る予定にしていたこともあって、夕食はホテルのビュッフェを予約していました。夕方から荒天になって屋外はどんどん風雨が強くなっていたので、予約をしていて良かったなと...,この 日 早朝 丸 一 日 展示 会 回る 予定 し い こと あっ 夕食 ホテル ビュッフェ 予約 し い 夕方 荒天 なっ 屋外 どんどん 風雨 強く なっ い 予約 し い 良かっ ...,この 日 早朝 丸 一 日 展示 会 回る 予定 し い こと あっ 夕食 ホテル ビュッフェ 予約 し い 夕方 荒天 なっ 屋外 どんどん 風雨 強く なっ い 予約 し い 良かっ ...
4,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),一休予約で100円引き！コスパいい！ホテルビュッフェは、旅行に行った気分になる！有明で降りたのは、はじめてで、楽しい！イチゴのモンブラン！自分で作る海鮮丼！桜...,一休予約で0円引き！コスパいい！ホテルビュッフェは、旅行に行った気分になる！有明で降りたのは、はじめてで、楽しい！イチゴのモンブラン！自分で作る海鮮丼！桜...,一休 予約 0 円 引き コス パ いい ホテル ビュッフェ 旅行 行っ 気分 なる 有明 降り の はじめて 楽しい イチゴ モンブラン 自分 作る 海鮮 丼 桜 ...,一休 予約 0 円 引き コス パ いい ホテル ビュッフェ 旅行 行っ 気分 なる 有明 降り の はじめて 楽しい イチゴ モンブラン 自分 作る 海鮮 丼 桜 ...


## ストップワード

In [77]:
# ストップワードの設定を行う関数を定義。今回はローカルのtxtファイルから設定できるようにした。
def set_stopwords():
    """
    Get stopwords from input document.
    """
    # Defined by SlpothLib
    slothlib_path = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    slothlib_file = urllib.request.urlopen(slothlib_path)
    slothlib_stopwords = [line.decode("utf-8").strip() for line in slothlib_file]
    slothlib_stopwords = [ss for ss in slothlib_stopwords if not ss==u'']
    
    stopwords_list = []
    


In [ ]:
# stopwords = set_stopwords()
# print(stopwords
# )

In [82]:
# リストを文字列に変換する関数
def join_list_str(list):
    return ' '.join(list)

# ストップワード除外関数
def exclude_stopword(text):
    changed_text = [token for token in text.lower().split(" ") if token != "" if token not in stopwords]
    # 上記のままだとリスト形式になってしまうため、空白区切の文字列に変換
    changed_text = join_list_str(changed_text)
    return changed_text

In [ ]:
# 飲食系口コミ分析に特化したストップワードリストの例
stop_words = [
    # 1. 口コミの定型的な表現
    '思う', '感じる', '美味しい', 'うまい', '普通', '良い', '悪い', '最高', '本当', 
    '今回', '前回', '初', '再訪', '人気', '有名', '定番', '満足', '好き', 
    'おすすめ', 'ごちそうさま', '様',

    # 2. 一般的すぎる名詞
    '料理', '食事', 'メニュー', 'ドリンク', '一品', 'ご飯',
     'コーヒー', '肉', '魚', '野菜',

    # 3. 一般的な動詞
    '食べる', '飲む', '行く', '来る', '入る', '出る', 'いる', 'ある', 'する', 
    'なる', '頼む', '注文', '訪問', '利用', '頂く', 'いただく', '会計',

    # 4. その他
    '店', 'お店', 'レストラン', 'こちら', 'ところ', '場所', '雰囲気', '店内', 
    '外観', '席', 'カウンター', 'テーブル', '個室', 'スタッフ', '店員', 
    '自分', '私', '人', '時間', '円', '的', 'こと', 'もの', 'これ', 'それ', 'ため',
    'さん', 'よう', '...',
]

stopwords = set(stop_words)

In [89]:
# 'stopwords'という名前のsetが既にある前提

def exclude_stopwords_from_wakachi(text):
    if not isinstance(text, str):
        return ''
    
    # テキストを空白で分割し、単語のリストにする
    words = text.split(' ')
    
    # ストップワードのsetに含まれていない単語だけを残す
    filtered_words = [word for word in words if word not in stopwords]
    
    # 再び空白で連結して文字列に戻す
    return ' '.join(filtered_words)

# --- データフレームへの適用例 ---
review_df['cleaned_text'] = review_df['review_clearn'].apply(exclude_stopwords_from_wakachi)
review_df.head()

,shop_name,review_text,review_number_to_zero,lsbw,review_clearn,cleaned_text
0,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),平日のランチブッフェに行きました。店内は広々していて、テーブルもイスも大きめで、ゆったり過ごせました。季節の料理がメインで、楽しく選ぶことができ、あまり混んでいなかったからか、並ぶこと...,平日のランチブッフェに行きました。店内は広々していて、テーブルもイスも大きめで、ゆったり過ごせました。季節の料理がメインで、楽しく選ぶことができ、あまり混んでいなかったからか、並ぶこと...,平日 ランチブッフェ 行き 店内 広々 し い テーブル イス 大きめ ゆったり 過ごせ 季節 料理 メイン 楽しく 選ぶ こと でき あまり 混ん い 並ぶ こと ...,平日 ランチブッフェ 行き 店内 広々 し い テーブル イス 大きめ ゆったり 過ごせ 季節 料理 メイン 楽しく 選ぶ こと でき あまり 混ん い 並ぶ こと ...,平日 ランチブッフェ 行き 広々 し い イス 大きめ ゆったり 過ごせ 季節 メイン 楽しく 選ぶ でき あまり 混ん い 並ぶ
1,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),GW中にレストラン利用のみで訪問。ホテルの中にあるため、エントランス、トイレなどを含めて全体的に綺麗で快適。今年はどうやら5周年の模様。料理の品数は可もなく不可もなく感じるが、一つ一つ丁寧...,GW中にレストラン利用のみで訪問。ホテルの中にあるため、エントランス、トイレなどを含めて全体的に綺麗で快適。今年はどうやら0周年の模様。料理の品数は可もなく不可もなく感じるが、一つ一つ丁寧...,GW 中 レストラン 利用 訪問 ホテル 中 ある ため エン トランス トイレ 含め 全体 的 綺麗 快適 今年 どうやら 0 周年 模様 料理 品数 可 なく 不可 なく 感じる 一つ 一つ 丁寧 ...,GW 中 レストラン 利用 訪問 ホテル 中 ある ため エン トランス トイレ 含め 全体 的 綺麗 快適 今年 どうやら 0 周年 模様 料理 品数 可 なく 不可 なく 感じる 一つ 一つ 丁寧 ...,GW 中 ホテル 中 エン トランス トイレ 含め 全体 綺麗 快適 今年 どうやら 0 周年 模様 品数 可 なく 不可 なく 一つ 一つ 丁寧
2,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),素敵なレストランでした️ライブキッチンでローストビーフとスープを頂けました。海鮮も置いてあり自由に丼が作れます。料理も充実していてとても良かったです。デザートにフルーツやケーキ...,素敵なレストランでした️ライブキッチンでローストビーフとスープを頂けました。海鮮も置いてあり自由に丼が作れます。料理も充実していてとても良かったです。デザートにフルーツやケーキ...,素敵 レストラン ライブ キッチン ロースト ビーフ スープ 頂け 海鮮 置い あり 自由 丼 作れ 料理 充実 し い とても 良かっ デザート フルーツ ケーキ ...,素敵 レストラン ライブ キッチン ロースト ビーフ スープ 頂け 海鮮 置い あり 自由 丼 作れ 料理 充実 し い とても 良かっ デザート フルーツ ケーキ ...,素敵 ライブ キッチン ロースト ビーフ スープ 頂け 海鮮 置い あり 自由 丼 作れ 充実 し い とても 良かっ デザート フルーツ ケーキ
3,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),この日は早朝から丸一日展示会を回る予定にしていたこともあって、夕食はホテルのビュッフェを予約していました。夕方から荒天になって屋外はどんどん風雨が強くなっていたので、予約をしていて良かったなと...,この日は早朝から丸一日展示会を回る予定にしていたこともあって、夕食はホテルのビュッフェを予約していました。夕方から荒天になって屋外はどんどん風雨が強くなっていたので、予約をしていて良かったなと...,この 日 早朝 丸 一 日 展示 会 回る 予定 し い こと あっ 夕食 ホテル ビュッフェ 予約 し い 夕方 荒天 なっ 屋外 どんどん 風雨 強く なっ い 予約 し い 良かっ ...,この 日 早朝 丸 一 日 展示 会 回る 予定 し い こと あっ 夕食 ホテル ビュッフェ 予約 し い 夕方 荒天 なっ 屋外 どんどん 風雨 強く なっ い 予約 し い 良かっ ...,この 日 早朝 丸 一 日 展示 会 回る 予定 し い あっ 夕食 ホテル ビュッフェ 予約 し い 夕方 荒天 なっ 屋外 どんどん 風雨 強く なっ い 予約 し い 良かっ
4,オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明),一休予約で100円引き！コスパいい！ホテルビュッフェは、旅行に行った気分になる！有明で降りたのは、はじめてで、楽しい！イチゴのモンブラン！自分で作る海鮮丼！桜...,一休予約で0円引き！コスパいい！ホテルビュッフェは、旅行に行った気分になる！有明で降りたのは、はじめてで、楽しい！イチゴのモンブラン！自分で作る海鮮丼！桜...,一休 予約 0 円 引き コス パ いい ホテル ビュッフェ 旅行 行っ 気分 なる 有明 降り の はじめて 楽しい イチゴ モンブラン 自分 作る 海鮮 丼 桜 ...,一休 予約 0 円 引き コス パ いい ホテル ビュッフェ 旅行 行っ 気分 なる 有明 降り の はじめて 楽しい イチゴ モンブラン 自分 作る 海鮮 丼 桜 ...,一休 予約 0 引き コス パ いい ホテル ビュッフェ 旅行 行っ 気分 有明 降り の はじめて 楽しい イチゴ モンブラン 作る 海鮮 丼 桜


In [91]:
import pandas as pd

# --- 事前準備：お手元の環境に合わせてください ---

# ★★★ こちらをお手元のデータフレーム名に書き換えてください ★★★
# この例では、処理済みのデータが 'review_df' という名前で存在すると仮定します。
# review_df = ... 

# --- ここからがCSV保存の処理 ---

# 1. 保存したい列を指定
columns_to_save = ['shop_name', 'cleaned_text']

# 2. 指定した列だけを抜き出して、新しいデータフレームを作成
output_df = review_df[columns_to_save]

# 3. 新しいCSVファイルとして保存
output_filename = 'odaiba_reviews_cleaned.csv'
output_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"\n処理完了「{output_filename}」にデータを保存しました")


処理完了「odaiba_reviews_cleaned.csv」にデータを保存しました
